In [162]:
!pip install gunicorn

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [33]:
import pandas as pd
import pydicom
from matplotlib import pyplot as plt
import png
import pydicom
import matplotlib.pyplot as plt
import scipy.misc
from skimage import io, transform, feature
from PIL import Image
import os
import re
import numpy as np

## Создаем корректные файлы с путями к файлам 

### Просмотр описания снимков

In [34]:
data = pd.read_csv('calc_case_description_test_set.csv')
data.head(10)

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_LEFT_CC_1/1.3.6.1.4.1.9590.1...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_LEFT_MLO_1/1.3.6.1.4.1.9590....
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_1/1.3.6.1.4.1.9590....
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/1.3.6.1.4.1.9590.10...,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....,Calc-Test_P_00038_RIGHT_CC_2/1.3.6.1.4.1.9590....
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_1/1.3.6.1.4.1.9590...
5,P_00038,2,RIGHT,MLO,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/1.3.6.1.4.1.9590.1...,Calc-Test_P_00038_RIGHT_MLO_2/1.3.6.1.4.1.9590...,Calc-Test_P_00038_RIGHT_MLO_2/1.3.6.1.4.1.9590...
6,P_00041,1,LEFT,CC,2,calcification,LUCENT_CENTER,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00041_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00041_LEFT_CC_2/1.3.6.1.4.1.9590.1...,Calc-Test_P_00041_LEFT_CC_2/1.3.6.1.4.1.9590.1...
7,P_00041,1,LEFT,MLO,2,calcification,LUCENT_CENTER,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00041_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00041_LEFT_MLO_2/1.3.6.1.4.1.9590....,Calc-Test_P_00041_LEFT_MLO_2/1.3.6.1.4.1.9590....
8,P_00077,2,LEFT,CC,1,calcification,ROUND_AND_REGULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Test_P_00077_LEFT_CC/1.3.6.1.4.1.9590.100...,Calc-Test_P_00077_LEFT_CC_1/1.3.6.1.4.1.9590.1...,Calc-Test_P_00077_LEFT_CC_1/1.3.6.1.4.1.9590.1...
9,P_00077,2,LEFT,MLO,1,calcification,ROUND_AND_REGULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,3,Calc-Test_P_00077_LEFT_MLO/1.3.6.1.4.1.9590.10...,Calc-Test_P_00077_LEFT_MLO_1/1.3.6.1.4.1.9590....,Calc-Test_P_00077_LEFT_MLO_1/1.3.6.1.4.1.9590....


### Изменяем пути к файлам изображений на верные

In [27]:
type_dir = ['F:\\CBIS-DDSM\\Calc-Test_', 'F:\\CBIS-DDSM\\Calc-Training_',  'F:\\CBIS-DDSM\\Mass-Test_', 'F:\\CBIS-DDSM\\Mass-Training_']
data_csv = ['calc_case_description_test_set.csv', 'calc_case_description_train_set.csv', 'mass_case_description_test_set.csv', 'mass_case_description_train_set.csv']

In [48]:
for sets in range(4):
    data = pd.read_csv(data_csv[sets])
    path_dir=type_dir[sets]
    for i in range(len(data)):
        path=str(path_dir)+str(data['patient_id'][i])+str('_')+str(data['left or right breast'][i])+str('_')+str(data['image view'][i])
        k=0
        for root, dirs, files in os.walk(path, topdown=True):
            k+=1
            if k==3:
                path_file=str(root)+str('\\')+str(files[0])
                file = path_file.replace('\\', '/')
                data.iloc[i, 11] = str(file[13:])
        if i==len(data)-1:
            print('Сохранено {} изображений из {}'.format(i+1, len(data)))
    data.to_csv('new_'+str(data_csv[sets]))

Сохранено 326 изображений из 326
Сохранено 1546 изображений из 1546
Сохранено 378 изображений из 378
Сохранено 1318 изображений из 1318


### Изменяем пути к файлам масок на верные

In [35]:
type_dir_mask = ['F:\\CBIS-DDSM\\Calc-Test_', 'F:\\CBIS-DDSM\\Calc-Training_',  'F:\\CBIS-DDSM\\Mass-Test_', 'F:\\CBIS-DDSM\\Mass-Training_']
data_csv_mask = ['new_calc_case_description_test_set.csv', 'new_calc_case_description_train_set.csv', 'new_mass_case_description_test_set.csv', 'new_mass_case_description_train_set.csv']

In [40]:
for j in range(4):
    data_mask = pd.read_csv(data_csv_mask[j])
    path_dir_mask=type_dir_mask[j]
    for i in range(len(data_mask)):
        path=str(path_dir_mask)+str(data_mask['patient_id'][i])+str('_')+str(data_mask['left or right breast'][i])+str('_')+str(data_mask['image view'][i])+str('_')+str(data_mask['abnormality id'][i])
        k=0
        for root, dirs, files in os.walk(path, topdown=True):
            k+=1
            if k==3:
                if len(files)==2:
                    path_file=str(root)+str('\\')+str(files[1])
                    file = path_file.replace('\\', '/')
                    data_mask.iloc[i, 13] = str(file[13:])
        if i==len(data_mask):
            print('Сохранено {} изображений из {}'.format(i+1, len(data_mask[j])))
    data_mask.to_csv('new_'+str(data_csv_mask[j]))

### Изменяем пути к файлам (обрезанным)

In [53]:
# datax = pd.read_csv('new_new_mass_case_description_train_set.csv')
# datax['ROI mask file path']=datax['cropped image file path']
# datax=datax.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
# datax.to_csv('masstrain.csv')

In [54]:
type_dir_crop = ['F:\\CBIS-DDSM\\Calc-Test_', 'F:\\CBIS-DDSM\\Calc-Training_',  'F:\\CBIS-DDSM\\Mass-Test_', 'F:\\CBIS-DDSM\\Mass-Training_']
data_csv_crop = ['calctest.csv', 'calctrain.csv', 'masstest.csv', 'masstrain.csv']

In [57]:
datax = pd.read_csv('calctest.csv').drop(columns=['Unnamed: 0'])

In [58]:
datax.head()

,patient_id,breast density,left or right breast,image view,abnormality id,abnormality type,calc type,calc distribution,assessment,pathology,subtlety,image file path,cropped image file path,ROI mask file path
0,P_00038,2,LEFT,CC,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_CC/08-29-2017-DDSM-9600...,Calc-Test_P_00038_LEFT_CC_1/08-29-2017-DDSM-94...,Calc-Test_P_00038_LEFT_CC_1/08-29-2017-DDSM-94...
1,P_00038,2,LEFT,MLO,1,calcification,PUNCTATE-PLEOMORPHIC,CLUSTERED,4,BENIGN,2,Calc-Test_P_00038_LEFT_MLO/08-29-2017-DDSM-176...,Calc-Test_P_00038_LEFT_MLO_1/08-29-2017-DDSM-1...,Calc-Test_P_00038_LEFT_MLO_1/08-29-2017-DDSM-1...
2,P_00038,2,RIGHT,CC,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/08-29-2017-DDSM-284...,Calc-Test_P_00038_RIGHT_CC_1/08-29-2017-DDSM-8...,Calc-Test_P_00038_RIGHT_CC_1/08-29-2017-DDSM-8...
3,P_00038,2,RIGHT,CC,2,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_CC/08-29-2017-DDSM-284...,Calc-Test_P_00038_RIGHT_CC_2/08-29-2017-DDSM-6...,Calc-Test_P_00038_RIGHT_CC_2/08-29-2017-DDSM-6...
4,P_00038,2,RIGHT,MLO,1,calcification,VASCULAR,NaN,2,BENIGN_WITHOUT_CALLBACK,5,Calc-Test_P_00038_RIGHT_MLO/08-29-2017-DDSM-85...,Calc-Test_P_00038_RIGHT_MLO_1/08-29-2017-DDSM-...,Calc-Test_P_00038_RIGHT_MLO_1/08-29-2017-DDSM-...


In [60]:
for j in range(4):
    data_crop = pd.read_csv(data_csv_crop[j]).drop(columns=['Unnamed: 0'])
    path_dir_crop=type_dir_crop[j]
    for i in range(len(data_crop)):
        path=str(path_dir_crop)+str(data_crop['patient_id'][i])+str('_')+str(data_crop['left or right breast'][i])+str('_')+str(data_crop['image view'][i])+str('_')+str(data_crop['abnormality id'][i])
        k=0
        for root, dirs, files in os.walk(path, topdown=True):
            k+=1
            if k==3:
                if len(files)==2:
                    path_file=str(root)+str('\\')+str(files[0])
                    file = path_file.replace('\\', '/')
                    data_crop.iloc[i, 13] = str(file[13:])
        if i==len(data_crop):
            print('Сохранено {} изображений из {}'.format(i+1, len(data_crop[j])))
    data_crop.to_csv('new_'+str(data_csv_crop[j]))

## Склеиваем calc и mass датасеты с новыми путями к файлам

In [120]:
new_data_csv = ['new_calctest.csv', 
                'new_calctrain.csv', 
                'new_masstest.csv', 
                'new_masstrain.csv']

In [121]:
data_test_1 = pd.read_csv(new_data_csv[0]).drop(columns=['Unnamed: 0'])
data_test_2 = pd.read_csv(new_data_csv[2]).drop(columns=['Unnamed: 0'])

data_test = pd.concat([data_test_1, data_test_2]).reset_index().drop(columns=['index'])

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [122]:
data_test = data_test.rename(columns={'cropped image file path':'ROI mask file path', 'ROI mask file path':'cropped image file path'})

In [123]:
len(data_test)

704

In [124]:
data_test.to_csv('_test.csv')

In [125]:
data_test['image file path'][1]

'Calc-Test_P_00038_LEFT_MLO/08-29-2017-DDSM-17613/1.000000-full mammogram images-97934/1-1.dcm'

In [126]:
data_train_1 = pd.read_csv(new_data_csv[1]).drop(columns=['Unnamed: 0'])
data_train_2 = pd.read_csv(new_data_csv[3]).drop(columns=['Unnamed: 0'])

data_train = pd.concat([data_train_1, data_train_2]).reset_index().drop(columns=['index'])

data_train.to_csv('_train.csv')

data_train = data_train.rename(columns={'cropped image file path':'ROI mask file path', 'ROI mask file path':'cropped image file path'})

data_train['image file path'][0]

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


'Calc-Training_P_00005_RIGHT_CC/08-07-2016-DDSM-23157/1.000000-full mammogram images-38548/1-1.dcm'

In [127]:
datax = pd.read_csv('_train.csv')

In [128]:
for i in range(len(datax)):
    datax['cropped image file path'][i]='F:\\CBIS-DDSM\\'+str(datax['cropped image file path'][i]).replace('/', '\\')
    datax['ROI mask file path'][i]='F:/CBIS-DDSM/'+str(datax['ROI mask file path'][i]).replace('/', '\\')
    datax['image file path'][i]='F:/CBIS-DDSM/'+str(datax['image file path'][i]).replace('/', '\\')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [129]:
datax.to_csv('train_xran.csv')

In [130]:
dataz = pd.read_csv('_test.csv')
for i in range(len(dataz)):
    dataz['cropped image file path'][i]='F:/CBIS-DDSM/'+str(dataz['cropped image file path'][i]).replace('/', '\\')
    dataz['ROI mask file path'][i]='F:/CBIS-DDSM/'+str(dataz['ROI mask file path'][i]).replace('/', '\\')
    dataz['image file path'][i]='F:/CBIS-DDSM/'+str(dataz['image file path'][i]).replace('/', '\\')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [131]:
dataz.to_csv('test_xran.csv')